Read in dataset and set curpath to working directory

In [63]:
import os
import sys
import time
import inspect
import numpy as np
import pandas as pd
from scipy.stats import skew
from scipy.stats import kurtosis

parentPath = '/'.join(sys.path[0].split('/'))

Create Spark Context & SQLContext

In [24]:
# Creating Spark Context
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

# Run the first time:
sc = SparkContext("local")

# Use to rerun script:
# sc = SparkContext.getOrCreate("local")
sqlContext = SQLContext(sc)
spark = SparkSession.builder.getOrCreate()

Load dataset from csv, using the "Expedia Hotel Recommendations" Kaggle dataset: https://www.kaggle.com/c/expedia-hotel-recommendations/overview

In [26]:
dfExpedia = spark.read.load(
  parentPath+'/data/train.csv',
  format="csv",
  sep=",",
  inferSchema=True,
  header=True
)

# Expose as SQL table
dfExpedia.createOrReplaceTempView('dfExpedia')

How many records in the training dataset?

In [27]:
dfExpedia.count()

37670293

How many columns in the training dataset?

In [28]:
len(dfExpedia.columns)

24

Take a 1% random sample of the training dataset

In [29]:
dfExpediaSample = dfExpedia.sample(withReplacement=False,fraction=0.01,seed=8)

How many records are in this sample?

In [30]:
dfExpediaSample.count()

375534

Preview the schema inferred when the dataset was read in

In [31]:
dfExpediaSample.printSchema()

root
 |-- date_time: timestamp (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable = true)
 |-- user_location_country: integer (nullable = true)
 |-- user_location_region: integer (nullable = true)
 |-- user_location_city: integer (nullable = true)
 |-- orig_destination_distance: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- is_mobile: integer (nullable = true)
 |-- is_package: integer (nullable = true)
 |-- channel: integer (nullable = true)
 |-- srch_ci: timestamp (nullable = true)
 |-- srch_co: timestamp (nullable = true)
 |-- srch_adults_cnt: integer (nullable = true)
 |-- srch_children_cnt: integer (nullable = true)
 |-- srch_rm_cnt: integer (nullable = true)
 |-- srch_destination_id: integer (nullable = true)
 |-- srch_destination_type_id: integer (nullable = true)
 |-- is_booking: integer (nullable = true)
 |-- cnt: integer (nullable = true)
 |-- hotel_continent: integer (nullable = true)
 |-- hotel_country: 

Convert Spark data.frame to Pandas data.frame

In [32]:
pd_dfExpediaSample = dfExpediaSample.toPandas()

Write sample data.frame to disk

In [33]:
pd_dfExpediaSample.to_csv(parentPath+'/data/pd___dfExpediaSample.csv', index=False)

In [67]:
import sys
import os
import inspect
import numpy as np
import pandas as pd
import unittest
import time

sys.path.insert(0, '/'.join(sys.path[0].split('/')[:-1]))

import src.dataPreparation as dp

In [69]:
tic = time.process_time()
parentPath = '/'.join(sys.path[0].split('/'))

df = pd.read_csv(parentPath+'/Expedia-Hotel-Prediction-Workflow/data/pd_dfExpediaSample.csv')

toc = time.process_time()
print("Elapsed time:", round(toc-tic, 3), "seconds")  

Elapsed time: 0.32 seconds


In [70]:
tic = time.process_time()

df = dp.updateIDFieldsToCategoricalFeatures(df)

toc = time.process_time()
print("Elapsed time:", round(toc-tic, 3), "seconds")  

Elapsed time: 0.02 seconds


In [71]:
tic = time.process_time()

df = dp.updateISFieldsToBooleanFeatures(df)

toc = time.process_time()
print("Elapsed time:", round(toc-tic, 3), "seconds")  

Elapsed time: 0.008 seconds


In [72]:
tic = time.process_time()

df = dp.removeHighCardinalityFeatures(df)

toc = time.process_time()
print("Elapsed time:", round(toc-tic, 3), "seconds")  

Elapsed time: 0.029 seconds


In [73]:
tic = time.process_time()

df = dp.removeHighNULLCntFeatures(df)

toc = time.process_time()
print("Elapsed time:", round(toc-tic, 3), "seconds")  

Elapsed time: 0.004 seconds


In [74]:
tic = time.process_time()

df = dp.removeRemainingRecordsWithNULLS(df)

toc = time.process_time()
print("Elapsed time:", round(toc-tic, 3), "seconds")  

Elapsed time: 0.06 seconds


In [75]:
tic = time.process_time()

df = dp.convertCategoricalVariablesToDummyVariables(df)

toc = time.process_time()
print("Elapsed time:", round(toc-tic, 3), "seconds")  

Elapsed time: 0.305 seconds


In [76]:
tic = time.process_time()

#pd_dfExpediaSample.to_csv(parentPath+'/Expedia-Hotel-Prediction-Workflow/data/pd_FullCleansedDataset.csv', index=False)

toc = time.process_time()

print("Elapsed time:", round(toc-tic, 3), "seconds")  

Elapsed time: 0.0 seconds


In [78]:
tic = time.process_time()

dfExpedia = spark.read.load(
  parentPath+'/Expedia-Hotel-Prediction-Workflow/data/train.csv',
  format="csv",
  sep=",",
  inferSchema=True,
  header=True
)

toc = time.process_time()

print("Elapsed time:", round(toc-tic, 3), "seconds")  

KeyboardInterrupt: 

In [ ]:
print("Elapsed time:", round(toc-tic, 3)/60, "minutes")  

In [60]:
dfExpedia.printSchema

<bound method DataFrame.printSchema of DataFrame[date_time: timestamp, site_name: int, posa_continent: int, user_location_country: int, user_location_region: int, user_location_city: int, orig_destination_distance: double, user_id: int, is_mobile: int, is_package: int, channel: int, srch_ci: timestamp, srch_co: timestamp, srch_adults_cnt: int, srch_children_cnt: int, srch_rm_cnt: int, srch_destination_id: int, srch_destination_type_id: int, is_booking: int, cnt: int, hotel_continent: int, hotel_country: int, hotel_market: int, hotel_cluster: int]>